In [1]:
"""
git clone https://github.com/MultiAgentLearning/playground.git
cd playground
pip install -e .
"""

import gym
import pommerman
from pommerman import agents

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

2024-05-18 08:32:50.055 Python[22617:1033192] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/rd/xnkdmhqx0c19zv0s0z9zcc9c0000gn/T/org.python.python.savedState
/Users/thakorns/Desktop/AltoTech/codebases/alto-academy-reinforcement-learning/playground/pommerman/network/client/__init__.py:155: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if match_obj[0] is 0:
/Users/thakorns/Desktop/AltoTech/codebases/alto-academy-reinforcement-learning/playground/pommerman/network/client/__init__.py:163: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif match_obj[0] is 2:


## 1. Explore Environment

In [10]:
agent_list = [
    agents.SimpleAgent(),
    agents.SimpleAgent(),
    agents.SimpleAgent(),
    agents.SimpleAgent()
]
env = pommerman.make('PommeFFACompetition-v0', agent_list)

In [12]:
env.action_space, env.observation_space

(Discrete(6), Box(372,))

In [13]:
env.reset()
env.render(mode='human')

In [18]:
actions = env.action_space.sample()
actions

3

In [46]:
actions = env.action_space.sample()
nobs, nreward, ndone, ninfo = env.step([actions]*4)
print(actions, nreward)
env.render(mode='human')

1 [-1, 0, 0, 0]


## 2. Learning Self-Play algorithm

In [48]:
# Create a custom environment for Pommerman
class PommermanEnv(gym.Env):
    def __init__(self):
        super(PommermanEnv, self).__init__()
        self.agent_list = [
            agents.SimpleAgent(),
            agents.SimpleAgent(),
            agents.SimpleAgent(),
            agents.SimpleAgent()
        ]
        self.env = pommerman.make('PommeFFACompetition-v0', self.agent_list)
        self.observation_space = self.env.observation_space[0]
        self.action_space = self.env.action_space[0]

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

    def reset(self):
        obs = self.env.reset()
        return obs

    def render(self, mode='human'):
        return self.env.render(mode=mode)

    def close(self):
        self.env.close()

In [1]:
# Initialize the custom Pommerman environment
def make_env():
    return PommermanEnv()

# Number of parallel environments
num_cpu = 4
env = SubprocVecEnv([make_env for _ in range(num_cpu)])

# Initialize the PPO model
model = PPO('MlpPolicy', env, verbose=1)

# Training with self-play
num_timesteps = int(1e6)
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./logs/',
                                         name_prefix='ppo_pommerman')

model.learn(total_timesteps=num_timesteps, callback=checkpoint_callback)

# Save the model
model.save("ppo_pommerman")

NameError: name 'SubprocVecEnv' is not defined

In [ ]:
# Load the model
model = PPO.load("ppo_pommerman")

# Evaluate the model
def evaluate(model, num_episodes=10):
    env = make_env()
    total_rewards = []
    for episode in range(num_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, info = env.step(action)
            total_reward += reward
        total_rewards.append(total_reward)
    avg_reward = sum(total_rewards) / num_episodes
    print(f'Average reward over {num_episodes} episodes: {avg_reward}')
    return avg_reward

evaluate(model)

# Play a game with the trained agent
env = make_env()
obs = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
env.close()